# Using OpenTSDB to Monitor Time-Series Data on Cloud Platform

### Task 1. Setup environment

In [ ]:
gcloud auth list
gcloud config list project

gcloud config set compute/zone us-central1-f

git clone https://github.com/GoogleCloudPlatform/opentsdb-bigtable.git

cd opentsdb-bigtable

### Task 2. Creating a Bigtable instance

In [ ]:
export BIGTABLE_INSTANCE_ID=bt-opentsdb
export ZONE=us-central1-f

gcloud bigtable instances create ${BIGTABLE_INSTANCE_ID} \
    --cluster-config=id=${BIGTABLE_INSTANCE_ID}-${ZONE},zone=${ZONE},nodes=1 \
    --display-name=OpenTSDB

### Task 3. Creating a Kubernetes Engine cluster

In [ ]:
gcloud container clusters create opentsdb-cluster \
    --zone=us-central1-f \
    --machine-type=n1-standard-4 \
    --scopes="https://www.googleapis.com/auth/cloud-platform"

### Task 4. Create the images used to test OpenTSDB

In [ ]:
export PROJECT_ID=$(gcloud config get project)
export REGION=us-central1
export AR_REPO=opentsdb-bt-repo
export BIGTABLE_INSTANCE_ID=bt-opentsdb
export ZONE=us-central1-f

gcloud artifacts repositories create ${AR_REPO} \
    --repository-format=docker  \
    --location=${REGION} \
    --description="OpenTSDB on bigtable container images"

### Task 5. Create the imaes used to deploy OpenTSDB

In [ ]:
export SERVER_IMAGE_NAME=opentsdb-server-bigtable
export SERVER_IMAGE_TAG=2.4.1

gcloud builds submit \
--tag ${REGION}-docker.pkg.dev/${PROJECT_ID}/${AR_REPO}/${SERVER_IMAGE_NAME}:${SERVER_IMAGE_TAG} \
build

export GEN_IMAGE_NAME=opentsdb-timeseries-generate
export GEN_IMAGE_TAG=0.1

cd generate-ts
./build-cloud.sh
cd ..

### Task 6. Create a ConfigMap

In [ ]:
envsubst < configmaps/opentsdb-config.yaml.tpl | kubectl create -f -

### Task 7. Create OpenTSDB tables in Bigtable

In [ ]:
envsubst < jobs/opentsdb-init.yaml.tpl | kubectl create -f -

kubectl describe jobs

OPENTSDB_INIT_POD=$(kubectl get pods \
    --selector=job-name=opentsdb-init \
    --output=jsonpath={.items..metadata.name})

kubectl logs $OPENTSDB_INIT_POD

### Task 8. Deploy OpenTSDB

In [ ]:
envsubst < deployments/opentsdb-write.yaml.tpl | kubectl create -f  -

envsubst < deployments/opentsdb-read.yaml.tpl | kubectl create -f  -

kubectl get pods

### Task 9. Create the OpenTSDB services

In [ ]:
kubectl create -f services/opentsdb-write.yaml

kubectl create -f services/opentsdb-read.yaml

kubectl get services

### Task 10. Writing to OpenTSDB

In [ ]:
envsubst < deployments/generate.yaml.tpl | kubectl create -f -

### Task 11. Examine the time series data with Open TDB

In [ ]:
kubectl create -f configmaps/grafana.yaml

kubectl get configmaps

kubectl create -f deployments/grafana.yaml

kubectl get deployments

GRAFANA_PODS=$(kubectl get pods --selector=app=grafana \
    --output=jsonpath={.items..metadata.name})

kubectl port-forward $GRAFANA_PODS 8080:3000